# Kaggle House Prices

### Import des librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import warnings
from scipy.stats import kurtosis, skew
from scipy import stats

%matplotlib inline
warnings.filterwarnings(action='once')

### Chargement des csv/datafiles :

In [2]:
trainfull = pd.read_csv('data/train.csv')
testfull = pd.read_csv('data/test.csv')

### 1ères informations

Les dimensions de nos datafiles:

In [3]:
print("trainfull : " + str(trainfull.shape))
print("testfull : " + str(testfull.shape))

trainfull : (1460, 81)
testfull : (1459, 80)


Soit :<br>
    1460 lignes (entrées) et 81 colonnes (features) pour le trainfull.<br>
    1459 lignes (entrées) et 80 colonnes (features) pour le testfull.<br>


In [4]:
trainfull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

1. On a donc 2 types de variables. Les variables **qualitatives**(*object*) et des **numériques** (*float64* et *int64*).<br><br>
2. Certaines variables ne sont pas pertinentes ou importantes. Les nombres de la 2ème colonne indique combien les features contiennent des éléments. Cela signifie qu'il ne compte que les éléments réels et élimine ceux qui contiennent des informations manquantes.<br>
Ici, on peut voir qu' `Alley`, `PoolQC` et `MiscFeature` ont respectivement que 91, 7 et 54 lignes remplies. On peut donc les considérer comme des features non importantes et les supprimer.<br>

Définissons un nouveau dataset`train` dans lequel le nombre total d'éléments réels dans les colonnes est supérieur ou égal à 30%.

In [5]:
train = trainfull[[column for column in trainfull if trainfull[column].count() / len(trainfull) >= 0.3]]

del train['Id']
print("List of dropped columns:", end=" ")
for c in trainfull.columns:
    if c not in train.columns:
        print(c, end=", ")
print('\n')

List of dropped columns: Id, Alley, PoolQC, Fence, MiscFeature, 



Nous devons faire la même chose pour le dataset `test`

In [6]:
test = testfull[[column for column in testfull if testfull[column].count() / len(testfull) >= 0.3]]

test_id = test['Id']

del test['Id']
print("List of dropped columns:", end=" ")
for c in testfull.columns:
    if c not in test.columns:
        print(c, end=", ")
print('\n')

List of dropped columns: Id, Alley, PoolQC, Fence, MiscFeature, 



In [7]:
print ("Size of train data : {}" .format(train.shape))
train.get_dtype_counts()
print ("Size of test data : {}" .format(test.shape))
test.get_dtype_counts()

Size of train data : (1460, 76)
Size of test data : (1459, 75)


int64      25
object     39
float64    11
dtype: int64

Il nous reste donc 76 features dont 37 numériques et 39 catégorielles.

On peut sauvegarder le dataset`train` en csv qui nous servira à l'exploration des données plus en profondeur.

Traitons maintenant les valeurs manquantes pour les autres features.

## Traitement des valeurs nulles - `Train`

In [8]:
# fonction pour les valeurs manquantes
def null_values(df):
    
    sum_null = df.isnull().sum()
    total = df.isnull().count()
    percent_nullvalues = 100* sum_null / total
    df_null = pd.DataFrame()
    df_null['Total'] = total
    df_null['Null_Count'] = sum_null
    df_null['Percent'] = round(percent_nullvalues,2)
    df_null['Type'] = df.loc[:, df.columns.isin(list(df.isnull()))].dtypes
    df_null = df_null.sort_values(by='Null_Count',ascending = False)
    df_null = df_null[df_null.Null_Count > 0]
    
    return(df_null)

In [9]:
null_values(train)

,Total,Null_Count,Percent,Type
FireplaceQu,1460,690,47.26,object
LotFrontage,1460,259,17.74,float64
GarageType,1460,81,5.55,object
GarageYrBlt,1460,81,5.55,float64
GarageFinish,1460,81,5.55,object
GarageQual,1460,81,5.55,object
GarageCond,1460,81,5.55,object
BsmtExposure,1460,38,2.60,object
BsmtFinType2,1460,38,2.60,object
BsmtQual,1460,37,2.53,object


On observe aussi qu'`Electrical` n'a qu'une seule valeur manquante. On peut la remplacer par la valeur qui se répète le plus.

In [10]:
print('Categories for Electrical variable:')
print(train['Electrical'].value_counts())

Categories for Electrical variable:
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64


On observe que la valeur `SBrkr` est la plus représentée. On remplace la valeur manquante par celle-ci.

In [11]:
train['Electrical'] = train['Electrical'].fillna('SBrkr')

c:\users\utilisateur\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Pour les features numériques nous allons remplacer les valeurs manquantes par *0*.<br>
Pour les features catégorielles nous allons remplacer les valeurs manquantes par *None*.

Variables quantitatives :

In [12]:
for col in ('LotFrontage', 'GarageYrBlt', 'MasVnrArea'):
    train[col] = train[col].fillna(0)

c:\users\utilisateur\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Variables qualitatives :

In [13]:
for col in ('FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
            'GarageCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtQual',
            'BsmtCond', 'BsmtFinType1', 'MasVnrType' ):
    train[col] = train[col].fillna('None')

c:\users\utilisateur\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [14]:
null_values(train)

,Total,Null_Count,Percent,Type


Nous n'avons plus aucune valeur nulle dans notre dataset.

# Traitement des valeurs nulles - `Test`

In [15]:
test_null = null_values(test)
test_null

,Total,Null_Count,Percent,Type
FireplaceQu,1459,730,50.03,object
LotFrontage,1459,227,15.56,float64
GarageQual,1459,78,5.35,object
GarageFinish,1459,78,5.35,object
GarageCond,1459,78,5.35,object
GarageYrBlt,1459,78,5.35,float64
GarageType,1459,76,5.21,object
BsmtCond,1459,45,3.08,object
BsmtExposure,1459,44,3.02,object
BsmtQual,1459,44,3.02,object


In [16]:
test_num = test.select_dtypes(exclude='object')
test_num = test_num.fillna(0)

In [17]:
test_quali = test.select_dtypes(include='object')
test_quali = test_quali.fillna('None')

In [18]:
test = pd.concat([test_quali, test_num], axis=1)

In [19]:
test.shape

(1459, 75)

In [20]:
test

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,730.0,140,0,0,0,120,0,0,6,2010
1,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,312.0,393,36,0,0,0,0,12500,6,2010
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,482.0,212,34,0,0,0,0,0,3,2010
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,470.0,360,36,0,0,0,0,0,6,2010
4,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,...,506.0,0,82,0,0,144,0,0,1,2010
5,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,...,440.0,157,84,0,0,0,0,0,4,2010
6,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,420.0,483,21,0,0,0,0,500,3,2010
7,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,393.0,0,75,0,0,0,0,0,5,2010
8,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,506.0,192,0,0,0,0,0,0,2,2010
9,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,525.0,240,0,0,0,0,0,0,4,2010


Nous avons à ce stade réaliser les mêmes traitements sur les datasets `train` et `test` qui sont :
    - supprimer les features inutiles (trop de valeurs manquantes)
    - traiter les valleurs nulles